# Importing libraries

In [141]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests
import json
from pandas.io.json import json_normalize

# Loading Data

In [149]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto_html=pd.read_html(url)
toronto_data=pd.DataFrame(toronto_html[0])
np.shape(toronto_data)

(288, 3)

In [150]:
toronto_data.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Data Cleaning

In [151]:
#drop cells where "Borough" is not assigned
toronto_data=toronto_data[toronto_data['Borough']!='Not assigned'].reset_index(drop=True)
toronto_data.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [152]:
#Regroup rows with similar postcodes
toronto_data=toronto_data.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index() #we group bu the postcode and Borough and then transform the last column
toronto_data.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [153]:
#changing 'Not asssigned' Neighbourhoods to Boroughs
not_assigned_index=toronto_data[toronto_data['Neighbourhood']=='Not assigned'].index.values.astype(int)[0] #index list of the rows where neighborhoods are not assigned 
toronto_data.where(toronto_data!='Not assigned',toronto_data['Borough'][not_assigned_index],inplace=True) #we replace these cells with their Borogh values
toronto_data.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [155]:
np.shape(toronto_data)

(103, 3)